In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [2]:
data = pd.read_feather("wide_data.feather")

X = data.loc[:, ~data.columns.isin(["HL_1", "HL_2", "HL_3", "HL_4", "HL_5", "HL_6"])]
y = data.loc[:, "HL_1":"HL_6"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1]:
mor_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=1000, random_state=42)).fit(X_train, y_train)

y_pred = mor_model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {rmse}")

NameError: name 'MultiOutputRegressor' is not defined

In [9]:
chain_model = RegressorChain(RandomForestRegressor(n_estimators=1000, random_state=42)).fit(X_train, y_train)

y_pred = chain_model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {rmse}")

Mean Squared Error: 2.2069133708892275
